In [21]:
import websocket
import json
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
import sympy as sp
import scipy as sc
from time import sleep
from binance import Client
client = Client(api_key="VQaW84v81fvM6b0z2cG97q0FBGK0VDVNYZGxPlMYkL1caao8u7lSrSWxeE5c6NKq",api_secret="9XgnO5nds9V8UVw5O8D0J6NHNPqL4ojj5t3FxZs7xqc8sd8MwEt44QshkT3W9auM",tld='com')

# from google.cloud.sql.connector import Connector
# from sqlalchemy import create_engine, text
# import pg8000.native

# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

# connector = Connector()

# def getconn() -> pg8000.native.Connection:
#     conn: pg8000.native.Connection = connector.connect(
#         "semiotic-pact-388619:us-central1:tradingbot",
#         "pg8000",
#         user="postgres",
#         password="GuruSat.3",
#         db="criptos"
#     )
#     return conn

# connection = create_engine('postgresql+pg8000://',creator=getconn)

In [24]:
read_account = client.get_account(recvWindow=35000)
read_account

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00100000',
  'taker': '0.00100000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1712739157047,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.00000119', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locke

In [ ]:
read_account

In [4]:
datetime.utcnow()

datetime.datetime(2024, 4, 9, 10, 34, 10, 247067)

In [14]:
import websocket
import json
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
import sympy as sp
import scipy as sc
from time import sleep
from binance import Client
client = Client(api_key="VQaW84v81fvM6b0z2cG97q0FBGK0VDVNYZGxPlMYkL1caao8u7lSrSWxeE5c6NKq",api_secret="9XgnO5nds9V8UVw5O8D0J6NHNPqL4ojj5t3FxZs7xqc8sd8MwEt44QshkT3W9auM",tld='com')

############################################################################################################################################################
# INICIO DEL BOT DE TRADING.
############################################################################################################################################################

##################################################
# PARÁMETROS INICIALES.
##################################################
###################################
# Parámetros de interpolación.
###################################
smooth_interval = 25
smooth_exp = 3

###################################
# Parámetros de estrategia.
###################################
rango = 1
std_mult = 110000
pdi_mult = 1000
sdi_mult = 200000
z_aprox = 0.1
pmax_idx = 0
pmin_idx = 0
last_time_buy = datetime.utcnow()
last_time_sell = datetime.utcnow()

###################################
# Metadatos de la estrategia.
###################################
estrategia = 'DERIVATIVES_TESTNET_V1'
trade_qty = 0.0001
k = 0
bullish = True
bearish = False

###################################
# Data Frames y Listas de Trabajo.
###################################
buy_orders = []
sell_orders = []
max_points_idx = []
min_points_idx = []
df_logs = pd.DataFrame(columns=['Tiempo','Accion'])

############################################################################################################################################################
# Ciclo anterior al streaming para obtener mejores LOGS y menos código innecesario.
#############################################################################################################################################################
while True:
    
    #############################################
    # Limpieza y almacenamiento de los datos de streaming en el df correspondiente.
    #############################################
    recent_data = pd.DataFrame(client.get_historical_klines('BTCUSDT','1m','3 hours ago UTC'))
    recent_data = recent_data.iloc[:,:2]
    recent_data.columns = ['Tiempo','Precio']
    recent_data['Tiempo'] = pd.to_datetime(recent_data.Tiempo,unit='ms')
    recent_data = recent_data.set_index('Tiempo')
    recent_data = recent_data.astype(float)
    recent_data = recent_data.reset_index()
    
    if k == 0:
        print(f'Inicia el bot de trading con inicio en: {datetime.utcnow()}')
        k += 1
    # else:
    #     if bullish == True:
    #         print(f'Se realizó una compra cerca del: {datetime.utcnow()}')
    #     elif bearish == True:
    #         print(f'Se realizó una venta cerca del: {datetime.utcnow()}')
    #         bearish = False

    ##################################################################################################################################
    # INICIA BOT DE TRADING: ACTUALIZACIÓN DE DATOS, CÁLCULO DE DERIVADAS, ESTRATEGIA Y TRADES AUTOMÁTICOS.
    ##################################################################################################################################    
    ##################################################################################################################################
    # CÁLCULO DE DERIVADAS, ESTRATEGIA Y TRADES AUTOMÁTICOS.
    ##################################################################################################################################        
    #############################################
    # CÁLCULO DE DERIVADAS
    #############################################
    recent_data['Interpolacion'] = sc.signal.savgol_filter(recent_data.Precio,smooth_interval,smooth_exp)
    recent_data['Primer_Derivada'] = recent_data.Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['PD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Primer_Derivada,smooth_interval,smooth_exp)
    recent_data['Segunda_Derivada'] = recent_data.PD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['SD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Segunda_Derivada,smooth_interval,smooth_exp)
    recent_data['Tercer_Derivada'] = recent_data.SD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['TD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Tercer_Derivada,smooth_interval,3)

    ######################################################################
    # Agregamos el código necesario para encontrar los puntos exactos mínimos y máximos
    ######################################################################
    try:
        initial_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo.iloc[:1].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() + rango
        final_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo.iloc[-1:].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() - rango
    except:
        initial_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo.iloc[:1].item()),'%Y-%m-%d %H:%M:%S')].index.item() + rango
        final_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo.iloc[-1:].item()),'%Y-%m-%d %H:%M:%S')].index.item() - rango

    lista_min_max = recent_data.iloc[initial_date_index:final_date_index][(recent_data.iloc[initial_date_index:final_date_index]['TD_Interpolacion'] >= -z_aprox) & (recent_data.iloc[initial_date_index:final_date_index]['TD_Interpolacion'] <= z_aprox)]
    indices_min_max = lista_min_max.index.tolist()
    mp = recent_data.Precio.iloc[initial_date_index:final_date_index].mean()

    pmin = 0
    pmax = 0

    #############################################
    # ESTRATEGIA
    #############################################
    for PMSD in indices_min_max:
        if PMSD > len(recent_data) - rango:
            break
        if (recent_data.SD_Interpolacion[PMSD:PMSD+1].item() > 0): # & (recent_data.PD_Interpolacion[PMSD:PMSD+1].item() > 0):
            pmax_idx = PMSD
            if (abs(recent_data.SD_Interpolacion[PMSD]*sdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD]*pdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                pmax = recent_data.SD_Interpolacion[PMSD]
            for i in range(rango+1):
                try:
                    if (i > PMSD) or (PMSD+i >= indices_min_max[-1:][0]):
                        break
                    elif ((recent_data.SD_Interpolacion[PMSD-i] > pmax)) & ((abs(recent_data.SD_Interpolacion[PMSD-i]*sdi_mult)) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD-i]*pdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmax_idx = PMSD-i
                    elif ((recent_data.SD_Interpolacion[PMSD+i] > pmax)) & ((abs(recent_data.SD_Interpolacion[PMSD+i]*sdi_mult)) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD+i]*pdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmax_idx = PMSD+i
                except:
                    pass
                try:
                    max_points_idx.append(pmax_idx)
                except:
                    pass
        elif (recent_data.SD_Interpolacion[PMSD:PMSD+1].item() < 0): # & (recent_data.PD_Interpolacion[PMSD:PMSD+1].item() < 0):
            pmin_idx = PMSD
            if (abs(recent_data.SD_Interpolacion[PMSD]*sdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD]*pdi_mult) < -abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                pmin = recent_data.SD_Interpolacion[PMSD]
            for i in range(rango+1):
                try:
                    if (i > PMSD) or (PMSD+i >= indices_min_max[-1:][0]):
                        break
                    elif ((recent_data.SD_Interpolacion[PMSD-i] < pmin)) & (abs(recent_data.SD_Interpolacion[PMSD-i]*sdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD-i]*pdi_mult) < -abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmin_idx = PMSD-i
                    elif ((recent_data.SD_Interpolacion[PMSD+i] < pmin)) & (abs(recent_data.SD_Interpolacion[PMSD+i]*sdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD+i]*pdi_mult) < -abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                        pmin_idx = PMSD+i
                except:
                    pass
                try:
                    min_points_idx.append(pmin_idx)
                except:
                    pass
    max_points_idx = list(set(max_points_idx))
    min_points_idx = list(set(min_points_idx))
    max_points_idx.sort()
    min_points_idx.sort()
    
    #############################################
    # AUTOMATIC TRADING.
    #############################################
    try:
        if bullish == False:
            
            if (recent_data.Tiempo[max_points_idx[-1]] != last_time_buy) & (recent_data.Tiempo[max_points_idx[-1]] > last_time_buy) & (recent_data.Tiempo[max_points_idx[-1]] > last_time_sell):
                last_time_buy = recent_data.Tiempo[max_points_idx[-1]]
                print(f'Precio de compra en: {recent_data.Precio[max_points_idx[-1]]}. Tiempo de compra: {recent_data.Tiempo[max_points_idx[-1]]}. Tiempo real de la señal: {datetime.utcnow()}. Diferencia entre señal y compra: {(datetime.utcnow() - recent_data.Tiempo[max_points_idx[-1]]).seconds} segundos.')
                lista_logs = [datetime.utcnow(),'C']
                df_logs.loc[len(df_logs)] = lista_logs
                print(max_points_idx[-1])
                marketorder_buy = client.create_order(
                    symbol='BTCUSDT',
                    side=Client.SIDE_BUY,
                    type=Client.ORDER_TYPE_MARKET,
                    quantity=trade_qty,
                    recvWindow=60000)
                bullish = True
                # bearish = False
                buy_orders.append(marketorder_buy)
                del marketorder_buy
                print(f'Punto de compra: {datetime.utcnow()}')
    except:
        pass
    try:
        if bullish == True:
            if (recent_data.Tiempo[min_points_idx[-1]] != last_time_sell) & (recent_data.Tiempo[min_points_idx[-1]] > last_time_sell) & (recent_data.Tiempo[min_points_idx[-1]] > last_time_buy):
                last_time_sell = recent_data.Tiempo[min_points_idx[-1]]
                print(f'Precio de venta en: {recent_data.Precio[min_points_idx[-1]]}. Tiempo de venta: {recent_data.Tiempo[min_points_idx[-1]]}. Tiempo real de la señal: {datetime.utcnow()}. Diferencia entre señal y compra: {(datetime.utcnow() - recent_data.Tiempo[min_points_idx[-1]]).seconds} segundos.')
                lista_logs = [datetime.utcnow(),'V']
                df_logs.loc[len(df_logs)] = lista_logs
                print(f'Punto de venta: {datetime.utcnow()}')
                print(min_points_idx[-1])
                marketorder_sell = client.create_order(
                    symbol='BTCUSDT',
                    side=Client.SIDE_SELL,
                    type=Client.ORDER_TYPE_MARKET,
                    quantity=trade_qty,
                    recvWindow=60000)
                sell_orders.append(marketorder_sell)
                bullish = False
                # bearish = True
                
    except:
        pass
    
    if (recent_data.Tiempo.iloc[-1:].item() >= datetime.strptime('2024-04-10 23:11:00','%Y-%m-%d %H:%M:%S')) & (bullish == False):
        break   


Inicia el bot de trading con inicio en: 2024-04-10 19:07:51.572034
Precio de venta en: 69332.0. Tiempo de venta: 2024-04-10 19:08:00. Tiempo real de la señal: 2024-04-10 19:11:00.695579. Diferencia entre señal y compra: 180 segundos.
Punto de venta: 2024-04-10 19:11:00.696583
176
Precio de compra en: 69248.64. Tiempo de compra: 2024-04-10 19:09:00. Tiempo real de la señal: 2024-04-10 19:12:00.502114. Diferencia entre señal y compra: 180 segundos.
176
Punto de compra: 2024-04-10 19:12:00.680431
Precio de venta en: 69263.99. Tiempo de venta: 2024-04-10 19:10:00. Tiempo real de la señal: 2024-04-10 19:13:00.637107. Diferencia entre señal y compra: 180 segundos.
Punto de venta: 2024-04-10 19:13:00.638614
176
Precio de compra en: 69240.13. Tiempo de compra: 2024-04-10 19:11:00. Tiempo real de la señal: 2024-04-10 19:14:00.474212. Diferencia entre señal y compra: 180 segundos.
176
Punto de compra: 2024-04-10 19:14:00.649787
Precio de venta en: 69250.0. Tiempo de venta: 2024-04-10 19:12:00. T

KeyboardInterrupt: 

In [10]:
marketorder_buy['orderId'],marketorder_buy['fills'][0]['price'],marketorder_buy['fills'][0]['qty'],marketorder_buy['workingTime'],marketorder_buy['fills'][0]['commission'],marketorder_buy['fills'][0]['tradeId']

(26377189486,
 '69091.60000000',
 '0.00010000',
 1712743172053,
 '0.00000010',
 3541434472)

In [7]:
marketorder_buy

{'symbol': 'BTCUSDT',
 'orderId': 26377189486,
 'orderListId': -1,
 'clientOrderId': 'iE2vIvDWxirbeCoP6OLxTu',
 'transactTime': 1712743172053,
 'price': '0.00000000',
 'origQty': '0.00010000',
 'executedQty': '0.00010000',
 'cummulativeQuoteQty': '6.90916000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'workingTime': 1712743172053,
 'fills': [{'price': '69091.60000000',
   'qty': '0.00010000',
   'commission': '0.00000010',
   'commissionAsset': 'BTC',
   'tradeId': 3541434472}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [ ]:
client.create_order(
                    symbol='BTCUSDT',
                    side=Client.SIDE_SELL,
                    type=Client.ORDER_TYPE_MARKET,
                    quantity=trade_qty,
                    recvWindow=60000)

In [ ]:
client.create_order(
                    symbol='BTCUSDT',
                    side=Client.SIDE_BUY,
                    type=Client.ORDER_TYPE_MARKET,
                    quantity=trade_qty,
                    recvWindow=60000)

In [15]:
from binance import Client
client = Client(api_key="VQaW84v81fvM6b0z2cG97q0FBGK0VDVNYZGxPlMYkL1caao8u7lSrSWxeE5c6NKq",api_secret="9XgnO5nds9V8UVw5O8D0J6NHNPqL4ojj5t3FxZs7xqc8sd8MwEt44QshkT3W9auM",tld='com')

client.get_account(recvWindow=50000) # BEFORE doing the strategy again: 1.- USDT: 9924.14895610, BTC: 1.00100000. 2.- USDT: 9924.12948920, BTC: 1.00100000.

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00100000',
  'taker': '0.00100000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1712777341956,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00017615', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.00000066', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locke

In [16]:
lista_de_ordenes = client.get_all_orders(symbol='BTCUSDT',recvWindow=40000)
len(lista_de_ordenes)

105

In [ ]:
for order in lista_de_ordenes:
    

In [3]:
len(sell_orders)

NameError: name 'sell_orders' is not defined

In [4]:
buy_orders

NameError: name 'buy_orders' is not defined

In [14]:
pmax_df = recent_data.iloc[max_points_idx].loc[:,['Tiempo','Precio']]
pmax_df['Accion'] = 'C'
pmin_df = recent_data.iloc[min_points_idx].loc[:,['Tiempo','Precio']]
pmin_df['Accion'] = 'V'
vc_df = pd.concat([pmax_df,pmin_df],axis=0)
vc_df = vc_df.reset_index()
vc_df = vc_df.set_index('index')
vc_df = vc_df.sort_index()
del_idx = []
# vc_df = vc_df.reset_index()
for i in vc_df.index:
    try:
        if vc_df.Accion[i+1] == vc_df.Accion[i]:
            del_idx.append(i+1)
    except:
        pass
vc_df.drop(del_idx,inplace=True)
vc_df = vc_df.reset_index(drop=True)
vc_df['Diferencia_Precio_USDT'] = pd.Series(dtype='float')
for i in range(len(vc_df)-1):
    vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 0.001
profits = 0
for i in range(len(vc_df)-1):
    if vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
profits

C:\Users\tinma\AppData\Local\Temp\ipykernel_31372\2802205712.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 0.001


0.5711799999999637

In [33]:
# recent_data = pd.read_csv('../../../csvs/streaming/BTCUSDT.csv')
# recent_data.columns = ['Tiempo','Simbolo','Precio']
# recent_data = recent_data.loc[:,['Tiempo','Precio']]
recent_data

,Tiempo,Precio


In [25]:
500 % 500 == 0

True

In [33]:
marketorder_buy = client.create_order(
                        symbol='XRPETH',
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.003,
                        recvWindow=10000)
marketorder_buy

# marketorder_buy ----> Diccionario: symbol, orderId, transactTime, orig_Qty, side, workingTime, fills ----> Lista con diccionarios. Keys: price, qty, comission, comissionaAsset, tradeId

{'symbol': 'XRPUSDT',
 'orderId': 4156737,
 'orderListId': -1,
 'clientOrderId': 'o4H8gQaUWCFIfR0cZgZlK0',
 'transactTime': 1712666256119,
 'price': '0.00000000',
 'origQty': '10.00000000',
 'executedQty': '10.00000000',
 'cummulativeQuoteQty': '6.36400000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'workingTime': 1712666256119,
 'fills': [{'price': '0.63640000',
   'qty': '10.00000000',
   'commission': '0.00000000',
   'commissionAsset': 'XRP',
   'tradeId': 266819}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [26]:
marketorder_sell = client.create_order(
                        symbol='ETHUSDT',
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.003,
                        recvWindow=10000)
marketorder_sell

{'symbol': 'ETHUSDT',
 'orderId': 8795137,
 'orderListId': -1,
 'clientOrderId': 'mGuWjb2npgk5KI8u2UdGVQ',
 'transactTime': 1712665283539,
 'price': '0.00000000',
 'origQty': '0.00300000',
 'executedQty': '0.00300000',
 'cummulativeQuoteQty': '10.88259000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1712665283539,
 'fills': [{'price': '3627.53000000',
   'qty': '0.00300000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 1818593}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [27]:
client.get_account(recvWindow=10000)

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1712665283539,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '0.99700000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.21963610', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '3798.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '727.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '27.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '91.0000000

In [ ]:
# balances = []
# ganancia_maxima = 0
# open_order = False
# balance_trade_anterior = 0
# balance_trade_p = 0
# tiempo_inicial = datetime.utcnow()
# #### PARÁMETROS DE LAS CONDICIONES DE SALIDA Y ENTRADA.
# PAIR = 'BTCUSDT'
# pair = 'btc'
# LOCALIDAD_SEÑAL_ENTRADA = 30 # EN SEGUNDOS
# # El primero de ellos es el promedio de la pendiente (precio/tiempo) de las últimas 10 filas del dataframe. Se utiliza para 
# # saber si se debe entrar en una posición corta al alza (puesto que a la baja no estamos tradeando) junto con otra condición
# PENDIENTE_ENTRADA = 0.02
# # LOS UMBRALES SON EN DÓLARES.
# UMBRAL_PERDIDA_1P = - 5
# UMBRAL_GANANCIA_1G = 20
# UMBRAL_GANANCIA_3G = 35
# UMBRAL_GANANCIA_4G = 5
# UMBRAL_MAXIMO_2G = 20
# UMBRAL_MINIMO_2G = 5
# LOCALIDAD_SEÑAL_SALIDA_2G = 30 # EN SEGUNDOS
# LIMITE_TIEMPO_POSITIVO_4G = 10 # EN MINUTOS
# log_path = f'../LOGS/{estrategia}.csv'
# df_logs = pd.DataFrame(columns=['Tiempo','Accion'])




# ##############  VERIFICAMOS SI NO HAY OPERACIONES ABIERTAS: Como solo vamos a trabajar con señales de entrada, toda posición
# ##############  está cerrada si la siguiente condición se cumple.

#     if open_order == False:

# ##############  EN CASO DE NO HABER, SE PROCEDE A VERIFICAR LA "LOCALIDAD" DE LA SEÑAL: En caso de ser una señal local se crea 
# ##############  una orden de compra y se inicia el proceso.

#         # pendientes = [0]

#         # for i in range(len(df)-1):
#         #     x = slopee(df.index[i],df.Precio[i],df.index[i+1],df.Precio[i+1])
#         #     pendientes.append(x)

#         # df['pendiente'] = pendientes

#         DIFF_SEÑAL_ENTRADA = (datetime.utcnow() - TIEMPO_SEÑAL_ENTRADA).total_seconds()

#         if DIFF_SEÑAL_ENTRADA < LOCALIDAD_SEÑAL_ENTRADA: # and df.pendiente[-10:-1].mean() > PENDIENTE_ENTRADA:
#             buy_order = client.create_order(
#                         symbol=PAIR,
#                         side=Client.SIDE_BUY,
#                         type=Client.ORDER_TYPE_MARKET,
#                         quantity=cantidad_trade,
#                         recvWindow=60000
#                     )
#             open_order = True
#             tiempo_entrada = datetime.utcnow()
#             precio_entrada = float(buy_order['fills'][0]['price'])
#             print(f'Se realizó una compra al precio: {precio_entrada}, en el momento: {tiempo_entrada}.')

# ##############  SI EXISTE UNA OPERACIÓN ABIERTA, LA SIGUIENTE CONDICIÓN SE CUMPLE: En ese caso, se debe estar atento a cuándo
# ##############  cerrar nuestra posición. Ya sea que estemos ganando o perdiendo; se establecen límites de pérdida y de ganancia,
# ##############  así como también casos en los que las señales de venta se toman en cuenta, tanto en pérdidas como en ganancias.

#     while open_order:

#         day = datetime.utcnow() - timedelta(seconds=1000)

#         with connection.connect() as conn:
#             df =conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
#         df = pd.DataFrame(df)[['Tiempo', 'Precio']].set_index('Tiempo')
#         df = df.sort_values(by='Tiempo')
#         MACD(df)
#         buy, sell = señal_MACD(df)

#         buyprices = df.Precio.iloc[buy]
#         sellprices = df.Precio.iloc[sell]
#         TIEMPO_SEÑAL_SALIDA = sellprices.index[-1]
#         precio_actual = df.Precio[-1]

#         profit = (precio_actual - precio_entrada)

#         if profit == 0:
#             continue 

# ##############  SEÑALES DE VENTA EN CASO DE TENER PÉRDIDAS.

# #############  La primera señal se enfoca en no superar el umbral de pérdidas.       

#         elif profit < 0:

#             if open_order:

#                 if profit < UMBRAL_PERDIDA_1P:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '1P'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)
                        
# ##############  SEÑALES DE VENTA EN CASO DE TENER GANANCIAS.

#         else:

# #############  La primera señal se enfoca en no perder todas las ganancias, en caso de existir ganancias. Se permite una pérdida
# #############  del 50% de la ganancia máxima.


#             if open_order:

#                 if profit > UMBRAL_GANANCIA_1G:

#                     if profit >= ganancia_maxima:
#                         ganancia_maxima = profit
#                     elif profit < ganancia_maxima:
#                         if (profit/ganancia_maxima - 1) < - 0.2: 
#                             sell_order = client.create_order(
#                                         symbol=PAIR,
#                                         side=Client.SIDE_SELL,
#                                         type=Client.ORDER_TYPE_MARKET,
#                                         quantity=cantidad_trade,
#                                         recvWindow=60000
#                                     )
#                             open_order = False
#                             Metodo = '1G'
#                             tiempo_salida = datetime.utcnow()
#                             precio_salida = float(sell_order['fills'][0]['price'])
#                             balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                             balances.append(balance_trade_p)

# #############  La segunda señal se enfoca en vender en caso de alcanzar ganancias mínimas del 0.07% y a la vez se da una señal
# #############  de venta.   

#             if open_order:     

#                 if profit <= UMBRAL_MAXIMO_2G and profit > UMBRAL_MINIMO_2G and (datetime.utcnow() - TIEMPO_SEÑAL_SALIDA).total_seconds() < LOCALIDAD_SEÑAL_SALIDA_2G:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '2G'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)
                    
# #############  La tercer señal se enfoca en vender en caso de alcanzar ganancias máximas del 1.4%

#             if open_order:
            
#                 if profit > UMBRAL_GANANCIA_3G:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '3G'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)

# ############# POR ÚLTIMO, VAMOS A GENERAR UNA VENTA SI EL TRADE ESTÁ EN GANANCIAS PERO NO SE HA VENDIDO DESPUÉS DE 10 MINUTOS.

#             if open_order:
            
#                 if ((datetime.utcnow() - tiempo_entrada).total_seconds()/60) > LIMITE_TIEMPO_POSITIVO_4G and profit > UMBRAL_GANANCIA_4G:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '4G'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)

######################## LOGS

    # if balance_trade_anterior != balance_trade_p:
    #     balance_trade_anterior = balance_trade_p
    #     if balance_trade_p>0:
    #         print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Ganancia de: {balance_trade_p} dólares. \nMétodo: {Metodo}')
    #         trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_p),Metodo,LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
    #                                     columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
    #         if os.path.exists(path+f'trades_ganados_{pair}.csv'):
    #             trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
    #                             mode='a',
    #                             index=False,
    #                             header=False
    #                             )
    #         else:
    #             trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
    #                             index=False,
    #                             header=True,
    #                             )
    #     elif balance_trade_p==0:
    #         print('No hubo pérdidas ni ganancias del trade')
    #     else:
    #         print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Pérdida de: {balance_trade_p} dólares. \nMétodo: {Metodo}')
    #         trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_p),Metodo,LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
    #                                     columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
    #         if os.path.exists(path+f'trades_perdidos_{pair}.csv'):
    #             trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
    #                             mode='a',
    #                             index=False,
    #                             header=False
    #                             )
    #         else:
    #             trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
    #                             index=False,
    #                             header=True,
    #                             )

In [3]:
path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/trade_logs/testnet/logs/Sesiones/MACD_MACD+logicv1/'

# sleep(5)
    
if open_order:
    sell_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=cantidad_trade
                        )
    open_order = False
    precio_salida = float(sell_order['fills'][0]['price'])
    balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
    balances.append(balance_trade_p)
    
if tiempo_entrada > tiempo_salida:
    tiempo_final = tiempo_entrada
elif tiempo_entrada < tiempo_salida:
    tiempo_final = tiempo_salida 

if sum(balances) > 0:
    session_log = pd.DataFrame([[len(balances), sum(balances), sum(balances)/len(balances), PAIR, estrategia, tiempo_inicial, tiempo_final,cantidad_trade]],
             columns=['Cantidad_Trades','Ganancia','Ganancia_Promedio_xTrade','PAR','Estrategia','Tiempo_inicial','Tiempo_final','Cantidad_trade'])
    if os.path.exists(path+'Sesiones_Ganadas.csv'):
        session_log.to_csv(path+'Sesiones_Ganadas.csv',
                        mode='a',
                        index=False,
                        header=False
                        )
    else:
        session_log.to_csv(path+'Sesiones_Ganadas.csv',
                        index=False,
                        header=True
                        )
elif sum(balances) < 0:
    session_log = pd.DataFrame([[len(balances), sum(balances), sum(balances)/len(balances), PAIR, estrategia, tiempo_inicial, tiempo_final,cantidad_trade]],
             columns=['Cantidad_Trades','Perdida','Ganancia_Promedio_xTrade','PAR','Estrategia','Tiempo_inicial','Tiempo_final','Cantidad_trade'])
    if os.path.exists(path+'Sesiones_Perdidas.csv'):
        session_log.to_csv(path+'Sesiones_Perdidas.csv',
                        mode='a',
                        index=False,
                        header=False
                        )
    else:
        session_log.to_csv(path+'Sesiones_Perdidas.csv',
                        index=False,
                        header=True
                        )

In [4]:
session_log

,Cantidad_Trades,Ganancia,Ganancia_Promedio_xTrade,PAR,Estrategia,Tiempo_inicial,Tiempo_final,Cantidad_trade
0,44,0.14105,0.003206,BTCUSDT,MACD_logicv1,2023-07-05 13:55:44.163789,2023-07-05 14:33:14.443281,0.005


In [12]:
# balances[-1] = balances[-1]*0.1
balances

[0.08279999999998836,
 0.07739999999997962,
 0.05650000000001455,
 -0.05930000000000291,
 0.057000000000007274,
 0.20440000000002329]

In [28]:
sum(balances) + 1.5547999999999957

-0.09275000000008782

In [4]:
client.get_account()

BinanceAPIException: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.

In [38]:
sum(balances)

-55.60899999998987

In [117]:
pd.to_datetime('2023-06-29 20:03:37.435685') - pd.to_datetime('2023-06-29 19:05:24.924680')

Timedelta('0 days 00:58:12.511005')

In [121]:
balances[-1]

-0.7869999999998982

In [39]:

buy_order = client.create_order(
                        symbol='BTCUSDT',
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1
                    )

In [32]:
sell_order = client.create_order(
                            symbol='BTCUSDT',
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=0.1
                        )

In [16]:
sell_order

{'symbol': 'BTCUSDT',
 'orderId': 10464206,
 'orderListId': -1,
 'clientOrderId': 'zmNXnF77mLQvDiTWZ94hZS',
 'transactTime': 1688165636752,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '3046.84891893',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1688165636752,
 'fills': [{'price': '30468.63000000',
   'qty': '0.00859600',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2990498},
  {'price': '30468.55000000',
   'qty': '0.06432900',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2990499},
  {'price': '30468.30000000',
   'qty': '0.02707500',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2990500}],
 'selfTradePreventionMode': 'NONE'}

In [5]:
marketorder_buy = client.create_order(
    symbol='BTCUSDT',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=0.1,
    recvWindow=10000)

In [16]:
marketorder_sell = client.create_order(
    symbol='BTCUSDT',
    side=Client.SIDE_SELL,
    type=Client.ORDER_TYPE_MARKET,
    quantity=0.1,
    recvWindow=10000)

In [17]:
limitorder_sell

{'symbol': 'BTCUSDT',
 'orderId': 15582800,
 'orderListId': -1,
 'clientOrderId': 'UbJC3pf8YldNgFrnFPYmys',
 'transactTime': 1712660072002,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '7057.64658920',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1712660072002,
 'fills': [{'price': '70620.43000000',
   'qty': '0.00574000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611441},
  {'price': '70620.30000000',
   'qty': '0.00390000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611442},
  {'price': '70618.14000000',
   'qty': '0.00100000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611443},
  {'price': '70614.61000000',
   'qty': '0.00100000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611444},
  {'price': '70613.49000000',
   'qty': '0.00393000',
   'commi

In [18]:
client.get_all_orders(symbol='BTCUSDT',recvWindow=10000) 15609234

order = client.get_order(
    symbol='BTCUSDT',
    orderId='15582800',
    recvWindow=10000)

In [19]:
order

{'symbol': 'BTCUSDT',
 'orderId': 15582800,
 'orderListId': -1,
 'clientOrderId': 'UbJC3pf8YldNgFrnFPYmys',
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '7057.64658920',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1712660072002,
 'updateTime': 1712660072002,
 'isWorking': True,
 'workingTime': 1712660072002,
 'origQuoteOrderQty': '0.00000000',
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [10]:
cancel_order = client.cancel_order(
    symbol='BTCUSDT',
    orderId='10459808')

BinanceAPIException: APIError(code=-2011): Unknown order sent.

In [15]:
limitorder

{'symbol': 'BTCUSDT',
 'orderId': 10459808,
 'orderListId': -1,
 'clientOrderId': 'Vgl1y7Y4unFF40r881ULls',
 'transactTime': 1688164704216,
 'price': '30800.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '3051.23838466',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'workingTime': 1688164704216,
 'fills': [{'price': '30512.38000000',
   'qty': '0.08717800',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 2989981},
  {'price': '30512.41000000',
   'qty': '0.01282200',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 2989982}],
 'selfTradePreventionMode': 'NONE'}

In [92]:
sell_order

{'symbol': 'BTCUSDT',
 'orderId': 9914874,
 'orderListId': -1,
 'clientOrderId': 'n3IbYFnRv5COlykPamCSvW',
 'transactTime': 1688061602684,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '3054.32647239',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1688061602684,
 'fills': [{'price': '30543.33000000',
   'qty': '0.02747100',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2793732},
  {'price': '30543.24000000',
   'qty': '0.05140300',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2793733},
  {'price': '30543.24000000',
   'qty': '0.02112600',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2793734}],
 'selfTradePreventionMode': 'NONE'}

In [20]:
client.get_account(recvWindow=10000)

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1712660072002,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.90000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '17057.64658920', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '3798.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '727.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '27.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '91.0000000

In [34]:
client.get_all_orders(symbol='BTCUSDT',recvWindow=10000) 15609234

[{'symbol': 'BTCUSDT',
  'orderId': 15582800,
  'orderListId': -1,
  'clientOrderId': 'UbJC3pf8YldNgFrnFPYmys',
  'price': '0.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.10000000',
  'cummulativeQuoteQty': '7057.64658920',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1712660072002,
  'updateTime': 1712660072002,
  'isWorking': True,
  'workingTime': 1712660072002,
  'origQuoteOrderQty': '0.00000000',
  'selfTradePreventionMode': 'EXPIRE_MAKER'},
 {'symbol': 'BTCUSDT',
  'orderId': 15609234,
  'orderListId': -1,
  'clientOrderId': 'E9ilsoPQPBOt3ypxACHUE7',
  'price': '0.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.10000000',
  'cummulativeQuoteQty': '7068.30954310',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1712663642250,
  'updateTime': 17126636